In [31]:
#intial init
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [33]:
# Specify the file path
file_path = 'CSV_files/NCHS_-_Drug_Poisoning_Mortality_by_County__United_States.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)
df.head()

,FIPS,Year,State,FIPS State,County,Population,Model-based Death Rate,Standard Deviation,Lower Confidence Limit,Upper Confidence Limit,Urban/Rural Category,Census Division
0,1001,2017,Alabama,1,"Autauga County, AL","55,448",11.08347,1.13357,9.08398,13.52386,Medium Metro,6
1,1001,2018,Alabama,1,"Autauga County, AL","55,533",10.58276,1.08240,8.67354,12.91299,Medium Metro,6
2,1001,2019,Alabama,1,"Autauga County, AL","55,769",11.05015,1.13016,9.05668,13.48319,Medium Metro,6
3,1001,2012,Alabama,1,"Autauga County, AL","54,970",6.75924,0.69168,5.53928,8.24839,Medium Metro,6
4,1001,2013,Alabama,1,"Autauga County, AL","54,747",7.12040,0.72859,5.83533,8.68899,Medium Metro,6


In [35]:
# Renaming the column "Model-based Death Rate" to "Death Rate"
df.rename(columns={"Model-based Death Rate": "Death Rate"}, inplace=True)
df.head()


,FIPS,Year,State,FIPS State,County,Population,Death Rate,Standard Deviation,Lower Confidence Limit,Upper Confidence Limit,Urban/Rural Category,Census Division
0,1001,2017,Alabama,1,"Autauga County, AL","55,448",11.08347,1.13357,9.08398,13.52386,Medium Metro,6
1,1001,2018,Alabama,1,"Autauga County, AL","55,533",10.58276,1.08240,8.67354,12.91299,Medium Metro,6
2,1001,2019,Alabama,1,"Autauga County, AL","55,769",11.05015,1.13016,9.05668,13.48319,Medium Metro,6
3,1001,2012,Alabama,1,"Autauga County, AL","54,970",6.75924,0.69168,5.53928,8.24839,Medium Metro,6
4,1001,2013,Alabama,1,"Autauga County, AL","54,747",7.12040,0.72859,5.83533,8.68899,Medium Metro,6


In [37]:
# First, convert the 'Population' column to numeric (removing commas)
df['Population'] = df['Population'].astype(str).str.replace(',', '').astype(int)


# Calculating the population-weighted average death rate for each state
state_aggregated = df.groupby('State')[['Death Rate', 'Population']].apply(
    lambda x: (x['Death Rate'] * x['Population']).sum() / x['Population'].sum()
).reset_index(name='Population-Weighted Death Rate')

# Displaying the resulting DataFrame with aggregated data by state
df.head()


,FIPS,Year,State,FIPS State,County,Population,Death Rate,Standard Deviation,Lower Confidence Limit,Upper Confidence Limit,Urban/Rural Category,Census Division
0,1001,2017,Alabama,1,"Autauga County, AL",55448,11.08347,1.13357,9.08398,13.52386,Medium Metro,6
1,1001,2018,Alabama,1,"Autauga County, AL",55533,10.58276,1.08240,8.67354,12.91299,Medium Metro,6
2,1001,2019,Alabama,1,"Autauga County, AL",55769,11.05015,1.13016,9.05668,13.48319,Medium Metro,6
3,1001,2012,Alabama,1,"Autauga County, AL",54970,6.75924,0.69168,5.53928,8.24839,Medium Metro,6
4,1001,2013,Alabama,1,"Autauga County, AL",54747,7.12040,0.72859,5.83533,8.68899,Medium Metro,6


In [43]:
# Calculating total deaths for each row using the given death rate and population
df['Total Deaths'] = (df['Death Rate'] / 100000) * df['Population']

# Aggregating the total deaths by state
total_deaths_per_state = df.groupby('State')['Total Deaths'].sum().reset_index(name='Total Deaths')

# Displaying the results
print(total_deaths_per_state.head())


        State  Total Deaths
0     Alabama  12262.176257
1      Alaska   2256.515299
2     Arizona  24638.641884
3    Arkansas   7007.826074
4  California  91501.038971


In [47]:
# Aggregating total population by state
state_population = df.groupby('State')['Population'].sum().reset_index(name='Total Population')

# Merging the total deaths and total population data
state_comparison = pd.merge(total_deaths_per_state, state_population, on='State')

# Displaying the comparison of total deaths and population by state
state_comparison = pd.merge(total_deaths_per_state, state_population, on='State')
print(state_comparison.head())


        State  Total Deaths  Total Population
0     Alabama  12262.176257          90814323
1      Alaska   2256.515299          13510750
2     Arizona  24638.641884         124591251
3    Arkansas   7007.826074          55501529
4  California  91501.038971         717040869


In [49]:
# Rounding the 'Total Population' and 'Total Deaths' columns to remove decimal points
state_comparison['Total Deaths'] = state_comparison['Total Deaths'].round(0).astype(int)
state_comparison['Total Population'] = state_comparison['Total Population'].round(0).astype(int)

# Displaying the updated DataFrame
print(state_comparison.head())


        State  Total Deaths  Total Population
0     Alabama         12262          90814323
1      Alaska          2257          13510750
2     Arizona         24639         124591251
3    Arkansas          7008          55501529
4  California         91501         717040869


In [63]:
# Recalculating the 'Percent Deaths' column in case it was not properly set
state_comparison_sorted['Percent Deaths'] = (state_comparison_sorted['Total Deaths'] / state_comparison_sorted['Total Population']) * 100
state_comparison_sorted['Percent Deaths'] = state_comparison_sorted['Percent Deaths'].round(2)

# Finding states with similar populations within a tolerance range (10%)
tolerance = 0.1  # 10% tolerance for similar population
similar_population_pairs = []

# Identifying pairs of states with similar populations
for i in range(len(state_comparison_sorted) - 1):
    state1 = state_comparison_sorted.iloc[i]
    state2 = state_comparison_sorted.iloc[i + 1]
    if abs(state1['Total Population'] - state2['Total Population']) / state1['Total Population'] <= tolerance:
        similar_population_pairs.append((state1, state2))

# Displaying high and low death rate states if pairs are found
if similar_population_pairs:
    # Sorting within the first pair based on 'Percent Deaths'
    sorted_pair = sorted(similar_population_pairs[0], key=lambda x: x['Percent Deaths'], reverse=True)
    similar_states_comparison = pd.DataFrame(sorted_pair)
    similar_states_comparison
else:
    "No states found with sufficiently similar populations within the specified tolerance range."

print(similar_population_pairs)




[(State                Vermont
Total Deaths            1905
Total Population    11878988
Percent Deaths          0.02
Name: 1, dtype: object, State               District of Columbia
Total Deaths                        3083
Total Population                12050832
Percent Deaths                      0.03
Name: 2, dtype: object), (State               North Dakota
Total Deaths                 911
Total Population        13409054
Percent Deaths              0.01
Name: 3, dtype: object, State                 Alaska
Total Deaths            2257
Total Population    13510750
Percent Deaths          0.02
Name: 4, dtype: object), (State               South Dakota
Total Deaths                1061
Total Population        15801377
Percent Deaths              0.01
Name: 5, dtype: object, State               Delaware
Total Deaths            4051
Total Population    17362667
Percent Deaths          0.02
Name: 6, dtype: object), (State                Montana
Total Deaths            2423
Total Populati